#### Verification using ResNext with GCL weights loaded using TIMM (HUGGING FACE) AND  ResNext with GCL weights loaded locally
The blocks will be self-contained. It will have all code it needs to make running and debugging easier. We do the feature extraction for two images in the query folder of the CPH city:
 x3vA7Bk0HNI6rGkDpDZQUQ.jpg (first, index 0 ) & SQNDJeXa8UQ9pHht-13PNg.jpg (seventh image, index 6)

In [1]:
import os
import sys
import timm
import torch
from tqdm import tqdm
from torchinfo import summary
import numpy as np
import torchvision.transforms as ttf
import numpy as np
import csv
# !pip install nbformat
%run model_notebook.ipynb

/home/peshmerge/anaconda3/envs/py3813/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
%%time

# Setup the paths and constants
BACKBONE = "resnext"
POOL = "GeM"
NORM = None
BATCH_SIZE = 1

# Choose the image you want to use (the corresponding query file will be loaded)
image_file_name = 'x3vA7Bk0HNI6rGkDpDZQUQ'
# image_file_name = 'SQNDJeXa8UQ9pHht-13PNg'

root_dir = 'global_feature_verification_dir/'
q_idx = os.path.join(root_dir,'cph',image_file_name+'_query.json')

# Convert any given npy features file to csv file
def convert_npy_features_to_csv(source_file, target_file):
    source_features = np.load(source_file)
    source_features_list = source_features.tolist()

    with open(target_file, "w") as f:
        wr = csv.writer(f)
        wr.writerows(source_features_list)

def extract_features(net, feats_file,f_length=2048):
    # To run the feature extraction just like nicola we need the following things:
    image_size = [480,640]
    image_t = ttf.Compose([ttf.Resize(size=(image_size[0],image_size[1])),
                        ttf.ToTensor(),
                        ttf.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
                        ])
    dl = create_dataloader("test",root_dir, q_idx,None,image_t, BATCH_SIZE)
    
    if not os.path.exists(feats_file):
        feats = np.zeros((len(dl.dataset), f_length))
        for i, batch in tqdm(enumerate(dl), desc="Extracting features"):
            x = net.forward(batch.cuda())
            feats[i * dl.batch_size:i * dl.batch_size + dl.batch_size] = x.cpu().detach().squeeze(0)
        np.save(feats_file, feats)
        print(f"{feats_file} has been saved..........")
    else:
        print(feats_file,"already exists. Skipping.")



## Extract Global features using using the model loaded from HuggigFace 
# model = timm.create_model("hf-hub:Raizusekku/siamese_network_gcl_trained")
# file_name_extension = '_using_timm_with'

## Extract Global features using using the model from the disk 
model_file_weights = os.path.join('generalized_contrastive_loss','Models','MSLS','MSLS_resnext_GeM_480_GCL.pth')
model = create_model(BACKBONE,POOL,norm=None,mode="single" )
file_name_extension = ''
try:
    model.load_state_dict(torch.load(model_file_weights)["model_state_dict"])
except:
    model.load_state_dict(torch.load(model_file_weights)["state_dict"])
model.eval()


if torch.cuda.is_available():
    model.cuda()

feats_file= os.path.join(root_dir,'cph',image_file_name+'_feature_file'+ file_name_extension+'.npy')
csv_file= os.path.join(root_dir,'cph',image_file_name+'_feature_file'+file_name_extension +'.csv')

extract_features(model,feats_file)

# We are here, it means the npy is saved, convert to csv file
convert_npy_features_to_csv(feats_file, csv_file)

Extracting features: 1it [00:00,  2.98it/s]

global_feature_verification_dir/cph/x3vA7Bk0HNI6rGkDpDZQUQ_feature_file_using_timm_with.npy has been saved..........
CPU times: user 1.17 s, sys: 245 ms, total: 1.41 s
Wall time: 2.03 s


# Summarize all models with and without parameters

In [8]:
import timm
from torchinfo import summary
import os
import timm
import torch
from tqdm import tqdm
from torchinfo import summary
import numpy as np
import torchvision.transforms as ttf
import numpy as np
import csv

# !pip install nbformat
%run model_notebook.ipynb

### Summary of the model from HuggingFace without any parameters (features_only= True, out_indices=(3,4))

In [12]:
model = timm.create_model("hf-hub:Raizusekku/siamese_network_gcl_trained")
model.cuda()
summary(model, input_size = (1, 3, 480, 640), col_names = ("input_size", "output_size", "num_params", "trainable"), verbose = 1, depth=5)

Layer (type:depth-idx)                   Input Shape               Output Shape              Param #                   Trainable
ResNet                                   [1, 3, 480, 640]          [1, 2048]                 --                        True
├─Conv2d: 1-1                            [1, 3, 480, 640]          [1, 64, 240, 320]         9,408                     True
├─BatchNorm2d: 1-2                       [1, 64, 240, 320]         [1, 64, 240, 320]         128                       True
├─ReLU: 1-3                              [1, 64, 240, 320]         [1, 64, 240, 320]         --                        --
├─MaxPool2d: 1-4                         [1, 64, 240, 320]         [1, 64, 120, 160]         --                        --
├─Sequential: 1-5                        [1, 64, 120, 160]         [1, 256, 120, 160]        --                        True
│    └─Bottleneck: 2-1                   [1, 64, 120, 160]         [1, 256, 120, 160]        --                        True
│    │ 

Layer (type:depth-idx)                   Input Shape               Output Shape              Param #                   Trainable
ResNet                                   [1, 3, 480, 640]          [1, 2048]                 --                        True
├─Conv2d: 1-1                            [1, 3, 480, 640]          [1, 64, 240, 320]         9,408                     True
├─BatchNorm2d: 1-2                       [1, 64, 240, 320]         [1, 64, 240, 320]         128                       True
├─ReLU: 1-3                              [1, 64, 240, 320]         [1, 64, 240, 320]         --                        --
├─MaxPool2d: 1-4                         [1, 64, 240, 320]         [1, 64, 120, 160]         --                        --
├─Sequential: 1-5                        [1, 64, 120, 160]         [1, 256, 120, 160]        --                        True
│    └─Bottleneck: 2-1                   [1, 64, 120, 160]         [1, 256, 120, 160]        --                        True
│    │ 

### Summary of the model from HuggingFace with all parameters (features_only= True, out_indices=(3,4))

In [11]:

model = timm.create_model("hf-hub:Raizusekku/siamese_network_gcl_trained", features_only=True,out_indices=(3,4))
model.cuda()
summary(model, input_size = (1, 3, 480, 640), col_names = ("input_size", "output_size", "num_params", "trainable"), verbose = 1, depth=5)

Layer (type:depth-idx)                   Input Shape               Output Shape              Param #                   Trainable
FeatureListNet                           [1, 3, 480, 640]          [1, 1024, 30, 40]         --                        True
├─Conv2d: 1-1                            [1, 3, 480, 640]          [1, 64, 240, 320]         9,408                     True
├─BatchNorm2d: 1-2                       [1, 64, 240, 320]         [1, 64, 240, 320]         128                       True
├─ReLU: 1-3                              [1, 64, 240, 320]         [1, 64, 240, 320]         --                        --
├─MaxPool2d: 1-4                         [1, 64, 240, 320]         [1, 64, 120, 160]         --                        --
├─Sequential: 1-5                        [1, 64, 120, 160]         [1, 256, 120, 160]        --                        True
│    └─Bottleneck: 2-1                   [1, 64, 120, 160]         [1, 256, 120, 160]        --                        True
│    │ 

Layer (type:depth-idx)                   Input Shape               Output Shape              Param #                   Trainable
FeatureListNet                           [1, 3, 480, 640]          [1, 1024, 30, 40]         --                        True
├─Conv2d: 1-1                            [1, 3, 480, 640]          [1, 64, 240, 320]         9,408                     True
├─BatchNorm2d: 1-2                       [1, 64, 240, 320]         [1, 64, 240, 320]         128                       True
├─ReLU: 1-3                              [1, 64, 240, 320]         [1, 64, 240, 320]         --                        --
├─MaxPool2d: 1-4                         [1, 64, 240, 320]         [1, 64, 120, 160]         --                        --
├─Sequential: 1-5                        [1, 64, 120, 160]         [1, 256, 120, 160]        --                        True
│    └─Bottleneck: 2-1                   [1, 64, 120, 160]         [1, 256, 120, 160]        --                        True
│    │ 

### Summary of the model ResNext +GCL weights from the local disk

In [13]:
model_file_weights = os.path.join('generalized_contrastive_loss','Models','MSLS','MSLS_resnext_GeM_480_GCL.pth')
model = create_model('resnext','GeM',norm=None,mode="single" )

try:
    model.load_state_dict(torch.load(model_file_weights)["model_state_dict"])
except:
    model.load_state_dict(torch.load(model_file_weights)["state_dict"])

model.eval()
model.cuda()
summary(model, input_size = (1, 3, 480, 640), col_names = ("input_size", "output_size", "num_params", "trainable"), verbose = 1, depth=10)

Using cache found in /home/peshmerge/.cache/torch/hub/facebookresearch_WSL-Images_main


Number of layers: 8
0 Conv2d IS TRAINED
1 BatchNorm2d IS TRAINED
2 ReLU IS TRAINED
3 MaxPool2d IS TRAINED
4 Sequential IS TRAINED
5 Sequential IS TRAINED
6 Sequential IS TRAINED
7 Sequential IS TRAINED
Layer (type:depth-idx)                        Input Shape               Output Shape              Param #                   Trainable
BaseNet                                       [1, 3, 480, 640]          [1, 2048]                 1                         Partial
├─Sequential: 1-1                             --                        --                        --                        True
│    └─Conv2d: 2-1                            [1, 3, 480, 640]          [1, 64, 240, 320]         9,408                     True
│    └─BatchNorm2d: 2-2                       [1, 64, 240, 320]         [1, 64, 240, 320]         128                       True
│    └─ReLU: 2-3                              [1, 64, 240, 320]         [1, 64, 240, 320]         --                        --
│    └─MaxPool2d: 

Layer (type:depth-idx)                        Input Shape               Output Shape              Param #                   Trainable
BaseNet                                       [1, 3, 480, 640]          [1, 2048]                 1                         Partial
├─Sequential: 1-1                             --                        --                        --                        True
│    └─Conv2d: 2-1                            [1, 3, 480, 640]          [1, 64, 240, 320]         9,408                     True
│    └─BatchNorm2d: 2-2                       [1, 64, 240, 320]         [1, 64, 240, 320]         128                       True
│    └─ReLU: 2-3                              [1, 64, 240, 320]         [1, 64, 240, 320]         --                        --
│    └─MaxPool2d: 2-4                         [1, 64, 240, 320]         [1, 64, 120, 160]         --                        --
│    └─Sequential: 2-5                        [1, 64, 120, 160]         [1, 256, 120, 160]   